Estatística e Modelos Probabilísticos 
====================
###### COE241 - 2017.2  
###### Lucas Vieira Gama - DRE 113080612  

Projeto de Curso
-----
Estudo probabilístico de um conjunto de dados.

## Introdução
Devido a atual relevância do mercado de crypto moedas no cenário mundial, foi escolhido um conjunto de dados histórios de diferentes modeas, disponível no site Kaggle (https://www.kaggle.com/sudalairajkumar/cryptocurrencypricehistory). Existem diferentes moedas e tokens disponíveis no mercado, e o objetivo deste trabalho é verificar se é possível encontrar algum padrão probabilístico que auxilie avaliação de preço e valor, e na consequente redução de riscos no investimento.

Para a execução dos experimentos e realização das análises, foi utilizada a linguagem de desenvolvimento Python, com o auxílio das ferramentas Jupyter Notebook, Numpy, Pandas, Matplotlib e Scipy.

### Dados
Os dados estão em diferentes datasets, que incluem as seguintes modeas:
* Bitcoin
* Ethereum
* Ripple
* Bitcoin cash
* Bitconnect
* Dash
* Ethereum Classic
* Iota
* Litecoin
* Monero
* Nem
* Neo
* Numeraire
* Stratis
* Waves

Para cada dataset, os dados estão organizados em série temporal diárias com as seguintes informações diárias:
* Date : data da observação
* Open : Preço de abertura no dia
* High : Maior valor atingido no dia
* Low : Menor valor atingido no dia
* Close : Preço de fechamento no dia
* Volume : Volume de transações no dia
* Market Cap : Total de capitalização no mercado USD

### Tratamento dos dados
Para trabalhar estatisticamente, havia o problema da variação do valor, que dependia do valor no dia anterior. Para simplificar e aproximar nosso modelo com as distribuições probabilísticas discutidas no curso, foi extraído a variação relativa diária, isto é, a diferença entre o preço de abertura e fechamento de cada dia, em porcentagem.

Para fins estatísticos, necessitamos de dados suficientemente grandes. Entretanto, algumas destas moedas possuem poucos meses de existência. Portanto, consideramos apenas as moedas criadas no máximo 1 ano atrás.

In [ ]:
import numpy as np # manipulação de matrizes
import pandas as pd # Manipulação de data/sets
import matplotlib.pyplot as plt # Para plotagem
import scipy.stats as stats # Normalizações, cálculos de densidade de probabilidade, etc
import math

In [ ]:
## Lendo o dataset e processando os dados
data = {}
data['BTC'] = pd.read_csv("data/bitcoin_price.csv")
data['ETH'] = pd.read_csv("data/ethereum_price.csv")
data['DSH'] = pd.read_csv("data/dash_price.csv")
data['LTC'] = pd.read_csv("data/litecoin_price.csv")
data['XMR'] = pd.read_csv("data/monero_price.csv")
data['NEM'] = pd.read_csv("data/nem_price.csv")
data['NEO'] = pd.read_csv("data/neo_price.csv")
data['OMG'] = pd.read_csv("data/omisego_price.csv")
data['XRP'] = pd.read_csv("data/ripple_price.csv")
data['STRAT'] = pd.read_csv("data/stratis_price.csv")
data['WAVES'] = pd.read_csv("data/waves_price.csv")

def getRelativeChange(ticker):
    return (ticker['Close'] - ticker['Open']) / ticker['Open']

variation = {}
for coin in data.keys():
    variation[coin] = data[coin].apply(getRelativeChange, axis=1)

### Histograma Normalizado
Com a mudança relativa no preço diário, foi possível traçar um histograma, para cada moeda. Podemos observar que ele possui uma distribuição selhemante a Distribuição Gaussiana / Normal.

In [ ]:
for coin in variation.keys():
    plt.hist(variation[coin], 100, normed=1)
    plt.xlabel('% Change')
    plt.ylabel('% Probability')
    plt.title(r'$\mathrm{Histogram\ of\ %s}$' % coin)
    plt.grid(True)
    plt.show()

### Média, Variância e Desvio Padrão Amostral
Com base no que foi descoberto, tentamos obter os parâmetros para análise e posteriormente plotagem da distribuição normal.

In [ ]:
totalVariation  = {}
mean            = {}
variance        = {}
std             = {}
for coin in variation.keys():
    print(coin)
    totalVariation[coin] = variation[coin].sum()
    mean[coin] = totalVariation[coin]/len(variation[coin])
    print("{0:15} {1}".format('Média', mean[coin]))

    # unbiased variance
    variance[coin] = variation[coin].var()
    print("{0:15} {1}".format('Variância', variance[coin]))

    std[coin] = (variance[coin])**(1/2)
    print("{0:15} {1}".format('Desvio Padrão', std[coin]))

Podemos observar, entretanto, que estes resultados não são significativos. Para o Bitcoin, por exemplo, o desvio padrão é 13x o valor da média.

### Distribuição Normal
Traçamos então o gráfico para as PDFs de cada moeda.

In [ ]:
for coin in variation.keys():
    # plots histogram
    plt.hist(variation[coin], 100, normed=1)
    plt.xlabel('% Change')
    plt.ylabel('% Probability')
    plt.title(r'$\mathrm{Normal\ Distribution\ of\ %s}$' % coin)
    
    # makes distribution
    rv = stats.norm(mean[coin], std[coin])
    
    # plots r.v.
    x = np.linspace(min(variation[coin]), max(variation[coin])) 
    plt.plot(x, rv.pdf(x), lw=2)
    plt.show()

Observamos visualmente que a Normal não aproxima bem o gráfico, que possui um pico mais acentuado em torno da média. Para verificar esta 

### Goodness of Fit - KS TTeste
Para avalior se os dados podem ser aproximados pelas distribuições, usamos o teste Goodness of Fit Kolmogorov-Smirnov para variáveis contínuas, e buscamos um erro $ \alpha $ de 5%.

In [ ]:
for coin in variation.keys():
    print('Coin %s' % coin)
    
    # statistic for alpha = 5%
    n = len(variation[coin])
    print('Statistic should be %f' % (1.35810/math.sqrt(n)))
    
    print(stats.kstest(variation[coin], 'norm', [mean[coin], std[coin]]))
    print('\n')

Como podemos observar, a estatística do teste com significância de 5% de  rejeitou as distribuições normais para todas as moedas. Necessitamos, portanto, de outra distribuição que melhor explique os dados.

### Distribuição T-Student
Com base na referência 1, que já estudou as variações das moedas em períodos anteriores, escolhemos a distribuição T-Student, por ter maior grau de liberdade e conseguir aproximar melhor o evento estudado, com uma cauda mais pesada.

In [ ]:
t_rv = {}
for coin in variation.keys():
    # plots histogram
    plt.hist(variation[coin], 100, normed=1)
    plt.xlabel('% Change')
    plt.ylabel('% Probability')
    plt.title(r'$\mathrm{Normal\ Distribution\ of\ %s}$' % coin)
    
    # estimates parameters with MLE
    df, location, scale = stats.t.fit(variation[coin])
    t_rv[coin] = stats.t(df, location, scale)
    
    # plots r.v.
    x = np.linspace(min(variation[coin]), max(variation[coin])) 
    plt.plot(x, t_rv[coin].pdf(x), lw=2)
    plt.show()

### Goodness of Fit - KS TTeste
Para avalior se os dados podem ser aproximados pelas distribuições, usamos o teste Goodness of Fit Kolmogorov-Smirnov para variáveis contínuas, e buscamos um erro $ \alpha $ de 5%.

In [ ]:
for coin in variation.keys():
    print('Coin %s' % coin)
    
    # statistic for alpha = 5%
    n = len(variation[coin])
    print('Statistic should be %f' % (1.35810/math.sqrt(n)))
    
    print(stats.kstest(variation[coin], t_rv[coin].cdf))
    print('\n')

Como podemos observar, as seguintes moedas foram aceitas como distribuidas por T Student:
* DSH
* LTC
* XMR
* NEM
* NEO
* OMG
* XRP
* STRAT
* WAVES

Com destaque para OMG e Waves, que tiveram os melhores resultados

### Q-Q Plot
Traçamos finalmente o gráfico Quantile Quantile Plot, para analisar visualmente a aproximação dos dados pela função de distribuição T-Student.

In [ ]:
for coin in variation.keys():
    stats.probplot(variation[coin], dist=t_rv[coin], plot=plt)
    plt.title("T Student Q-Q plot %s " % coin)
    plt.show()

### Referências
1 Jörg Osterrieder. The Statistics of Bitcoin and Cryptocurrencies. Novembro, 2017.  
  
2 Stephen Chan, Jeffrey Chu, Saralees Nadarajah, Joerg Osterrieder. A Statistical Analysis of Cryptocurrencies. Maio 2017.
### Anexos
O código deste trabalho está disponível em https://github.com/lucasvg/crypoCurrencies-estModProb-COE241-UFRJ